In [40]:
import os
import argparse

import cv2
import pycuda.autoinit  # This is needed for initializing CUDA driver

from utils.yolo_classes import get_cls_dict
from utils.visualization import BBoxVisualization
from utils.yolo_with_plugins import TrtYOLO
from pathlib import Path
from matplotlib import pyplot as plt
import time
from tqdm import tqdm as tqdm_pb
import numpy as np

In [44]:
people_model_416 = {
    "names": Path("./blurring/common/yolov4-tiny.names").read_text().split(),
    "trt_path": "./blurring/common/416/yolov4-tiny.trt"
}
people_model_640 = {
    "names": Path("./blurring/common/yolov4-tiny.names").read_text().split(),
    "trt_path": "./blurring/common/640/yolov4-tiny.trt"
}

lp_model_fullhd = {
    "names": ["lp"],
    "trt_path": "./blurring/lp/fullhd/yolov4.trt"
}
lp_model_960 = {
    "names": ["lp"],
    "trt_path": "./blurring/lp/960/yolov4.trt"
}


In [58]:
people_model = people_model_416
plates_model = lp_model_960
trt_yolo_people = TrtYOLO(people_model["trt_path"], len(people_model["names"]), True)
trt_yolo_plates = TrtYOLO(plates_model["trt_path"], len(plates_model["names"]), True)

[04/04/2024-08:31:24] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.

[04/04/2024-08:31:24] [TRT] [I] Loaded engine size: 12 MiB
[04/04/2024-08:31:24] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 991, GPU 10005 (MiB)
[04/04/2024-08:31:24] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +0, now: CPU 991, GPU 10005 (MiB)
[04/04/2024-08:31:24] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +1, GPU +11, now: CPU 1, GPU 562 (MiB)
[04/04/2024-08:31:24] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 979, GPU 10005 (MiB)
[04/04/2024-08:31:24] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +0, now: CPU 979, GPU 10005 (MiB)
[04/04/2024-08:31:24] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContex

In [54]:
video_path = "/home/ubuntu/code/rv_data/data/nysta/front/short/output.mp4"
cap = cv2.VideoCapture(video_path)

In [59]:
_, frame = cap.read()
times = []

for _ in tqdm_pb(range(256), total=256):
    t1 = time.perf_counter()
    boxes, confs, clss = trt_yolo_people.detect(frame)
    boxes, confs, clss = trt_yolo_plates.detect(frame)
    times.append(time.perf_counter() - t1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:11<00:00, 21.40it/s]


In [60]:
1 / np.mean(np.array(times))

21.679259963893575

In [ ]:
times
